<a href="https://colab.research.google.com/github/BaduA/carpriceprediction/blob/main/carprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import Normalization

**Data Preperation**

In [ ]:
data = pd.read_csv("train.csv",",")
data.head()

In [ ]:
data.shape

(1000, 12)

In [ ]:
tensor_data = tf.constant(data)
tensor_data = tf.random.shuffle(tensor_data)


In [ ]:
X = tensor_data[:,3:-1]
Y = tensor_data[:,-1]
Y = tf.expand_dims(Y, axis = -1)

In [ ]:
normalizer = Normalization()
normalizer.adapt(X)
normalizer(X)[:5]

In [ ]:
model = tf.keras.Sequential([normalizer,Dense(1)])